In [ ]:
import numpy as np
import tensorflow as tf 
import librosa
import pandas as pd

class CNNModel:
  def __init__(self):
    std = np.sqrt(2) * np.sqrt(2.0 / ((N_CHANNELS + N_FILTERS) * 11))
    kernel = np.random.randn(1, 11, N_CHANNELS, N_FILTERS)*std
    kernel = tf.constant(kernel, name = "kernel", dtype = "float32")
    # not sure if this is the correct way to go for 1d networks... 
    self.cnn1 = tf.nn.conv1d(input=1025,, kernel, stride=1, padding=1)




def get_style_model_and_losses(cnn, style_float,style_weight=style_weight, style_layers=style_layers_default): #STYLE WEIGHT
		
		cnn = copy.deepcopy(cnn)
		style_losses = []
		model = tf.keras.Sequential()  # the new Sequential module network
		gram = GramMatrix()  # we need a gram module in order to compute style targets
		if tf.test.is_gpu_available():  
			model = model.cuda() # ---- Apparently there's no function for this command in Tensorflow and the gpu has to be pre-set in Tensorflow
      gram = gram.cuda()

    # name = tf.Module('conv_1')
		# model.add_module(name, cnn.cnn1)
		if name in style_layers:
			target_feature = model(style_float).clone()
			target_feature_gram = gram(target_feature)
			style_loss = StyleLoss(target_feature_gram, style_weight)
			# model.add_module("style_loss_1", style_loss)
			style_losses.append(style_loss)

		#name = 'pool_1'
		#model.add_module(name, cnn.pool1)

		'''name = 'fc_1'
		model.add_module(name, cnn.fc1)
		name = 'nl_9'
		model.add_module(name, cnn.nl9)
		name = 'fc_2'
		model.add_module(name, cnn.fc2)'''

		return model, style_losses


	input_float = content_float.clone()
	#input_float = Variable(torch.randn(content_float.size())).type(torch.FloatTensor)

	learning_rate_initial = 0.03

	def get_input_param_optimizer(input_float):
		input_param = nn.Parameter(input_float.data)    # line------
		#optimizer = optim.Adagrad([input_param], lr=learning_rate_initial, lr_decay=0.0001,weight_decay=0)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_initial, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
		return input_param, optimizer

	num_steps= 2500

	def run_style_transfer(cnn, style_float, input_float, num_steps=num_steps, style_weight=style_weight): #STYLE WEIGHT, NUM_STEPS
		print('Building the style transfer model..')
		model, style_losses= get_style_model_and_losses(cnn, style_float, style_weight)
		input_param, optimizer = get_input_param_optimizer(input_float)
		print('Optimizing..')
		run = [0]

		while run[0] <= num_steps:
			def closure():
            	# correct the values of updated input image
				input_param.data = tf.clip_by_value(input_param.data, clip_value_min=0, clip_value_max=1)
				model(input_param)
				style_score = 0

				for sl in style_losses:
					#print('sl is ',sl,' style loss is ',style_score)
					style_score += sl.backward()

				run[0] += 1
				if run[0] % 100 == 0:
					print("run {}:".format(run))
					print('Style Loss : {:8f}'.format(style_score.data[0])) #CHANGE 4->8 
					print()

				return style_score

		input_param.data = tf.clip_by_value(input_param.data, clip_value_min=0, clip_value_max=1)
		return input_param.data
		
	output = run_style_transfer(cnn, style_float, input_float)

	#output = output.squeeze(0)
	output = output.squeeze(0) tf.test.is_gpu_available()
	output = output.numpy()
	#print(output.shape)
	#output = output.resize([1025,2500])
	
	N_FFT=2048
	a = np.zeros_like(output)
	a = np.exp(output) - 1

	# This code is supposed to do phase reconstruction
	p = 2 * np.pi * np.random.random_sample(a.shape) - np.pi
	for i in range(500):
		S = a * np.exp(1j*p)
		x = librosa.istft(S)
		p = np.angle(librosa.stft(x, N_FFT))

	OUTPUT_FILENAME = 'output1D_4096_iter'+str(num_steps)+'_c'+content_audio_name+'_s'+style_audio_name+'_sw'+str(style_weight)+'_k3s1p1.wav'
	librosa.output.write_wav(OUTPUT_FILENAME, x, style_sr)

	print('DONE...')